In [4]:
import os
import polars as pl

In [5]:
current_dir = os.getcwd()
scripts = os.path.dirname(current_dir)
parent_dir = os.path.dirname(scripts)
output_path = os.path.join(parent_dir, "data", "tennis_dataset_app.csv")

In [6]:
dataset_path = os.path.join(parent_dir, "data", "tennis_dataset_clean_essaie.csv")
df = pl.read_csv(dataset_path)
df = df.with_columns(
    pl.col("date").str.strptime(pl.Date, format="%d.%m.%y").alias("date")
)
df

index,player1_name,player1_age,player1_ranking,player1_points,player1_win_rate,player1_win_rate_3_sets,player1_win_rate_tiebreak,player1_total_matches_dure,player1_win_rate_dure,player1_total_matches_terre battue,player1_win_rate_terre battue,player1_total_matches_gazon,player1_win_rate_gazon,player1_total_matches_salle,player1_win_rate_salle,player1_total_matches_carpet,player1_win_rate_carpet,player1_total_matches_acryl,player1_win_rate_acryl,player1_avg_first_serve_pct,player1_avg_first_serve_won_pct,player1_avg_second_serve_won_pct,player1_avg_return_points_won_pct,player1_avg_break_point_won_pct,player1_avg_double_fautes,player1_avg_aces,player2_name,player2_age,player2_ranking,player2_points,player2_win_rate,player2_win_rate_3_sets,player2_win_rate_tiebreak,player2_total_matches_dure,player2_win_rate_dure,player2_total_matches_terre battue,player2_win_rate_terre battue,player2_total_matches_gazon,player2_win_rate_gazon,player2_total_matches_salle,player2_win_rate_salle,player2_total_matches_carpet,player2_win_rate_carpet,player2_total_matches_acryl,player2_win_rate_acryl,player2_avg_first_serve_pct,player2_avg_first_serve_won_pct,player2_avg_second_serve_won_pct,player2_avg_return_points_won_pct,player2_avg_break_point_won_pct,player2_avg_double_fautes,player2_avg_aces,tournament_category,target,date,surface_dure,surface_salle,surface_terre_batue,surface_gazon,ranking_diff,points_diff,win_rate_diff,win_rate_diff_3_sets,win_rate_tiebreak_diff,aces_diff,double_faults_diff,win_rate_dure_diff,win_rate_terre_diff,win_rate_gazon_diff,win_rate_salle_diff
i64,str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,i64,f64,i64,f64,i64,f64,i64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,i64,date,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""Lucas Pouille""",30,101,616,0.675676,0.6,0.769231,12,0.75,13,0.615385,6,0.833333,6,0.5,0,0,0,0,0.569359,0.740458,0.53238,0.38835,0.405622,3.166667,6.833333,"""Gijs Brouwer""",28,176,326,0.565217,0.666667,0.375,18,0.555556,11,0.545455,12,0.583333,5,0.6,0,0,0,0,0.684263,0.699219,0.512169,0.385107,0.382979,1.5,3.826087,2,0,2024-09-22,1,0,0,0,75,290,0.110458,0.066667,0.0,3.007246,1.666667,0.194444,0.06993,0.25,0.1
1,"""Taylor Harry Fritz""",27,4,5100,0.75,0.727273,0.75,0,0.0,8,0.625,12,0.833333,0,0.0,0,0,0,0,0.634857,0.799842,0.559835,0.348439,0.4,1.590909,12.863636,"""Jack Draper""",23,15,2685,0.56,0.466667,0.555556,4,0.25,11,0.454545,10,0.8,0,0.0,0,0,0,0,0.604895,0.792733,0.493047,0.360829,0.395833,3.24,9.08,3,1,2024-07-30,0,0,1,0,11,2415,0.19,0.260606,0.0,3.783636,1.649091,0.25,0.170455,0.033333,0.0
2,"""Tommy Paul""",27,12,3145,0.711111,0.576923,0.916667,18,0.611111,14,0.714286,12,0.833333,1,1.0,0,0,0,0,0.616502,0.713651,0.555318,0.412325,0.397849,1.733333,5.177778,"""Miomir Kecmanovic""",25,54,1021,0.45,0.444444,0.461538,17,0.294118,12,0.5,8,0.5,3,1.0,0,0,0,0,0.622523,0.710345,0.529309,0.359486,0.391667,2.102564,4.230769,1,1,2024-10-18,0,1,0,0,42,2124,0.261111,0.132479,0.0,0.947009,0.369231,0.316993,0.214286,0.333333,0.0
3,"""Holger Rune""",21,13,3025,0.625,0.625,0.666667,18,0.611111,17,0.647059,5,0.6,0,0.0,0,0,0,0,0.596603,0.724453,0.542105,0.349387,0.396552,3.075,5.775,"""Nicolas Jarry""",29,35,1370,0.454545,0.5,0.6,19,0.421053,18,0.5,1,0.0,6,0.5,0,0,0,0,0.632006,0.746787,0.507726,0.329688,0.334802,2.272727,9.022727,2,1,2024-10-22,0,1,0,0,22,1655,0.170455,0.125,0.0,3.247727,0.802273,0.190058,0.147059,0.6,0.5
5,"""Damir Dzumhur""",32,83,679,0.678571,0.642857,0.5,5,0.6,18,0.777778,5,0.4,0,0.0,0,0,0,0,0.685901,0.639356,0.523962,0.432635,0.445344,2.285714,0.535714,"""Jesper De Jong""",24,112,548,0.71875,0.571429,0.857143,6,0.833333,24,0.708333,2,0.5,0,0.0,0,0,0,0,0.564008,0.730239,0.513133,0.416467,0.442379,3.3125,5.53125,2,1,2024-09-07,1,0,0,0,29,131,0.040179,0.071429,0.0,4.995536,1.026786,0.233333,0.069444,0.1,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [7]:
joueurs = set(list(df.select(pl.col("player1_name")).to_numpy().flatten()) + list(df.select(pl.col("player2_name")).to_numpy().flatten()))
assert len(joueurs) == 200, f"nombre de joueurs incomplet, joueurs = {len(joueurs)}"

In [8]:
derniers_matches = pl.DataFrame()
for joueur in joueurs:
    dernier_match = (
        df.filter(
            (pl.col("player1_name") == joueur) | (pl.col("player2_name") == joueur)
        )
        .sort(by="date", descending=True)
        .head(1)
    )
    if derniers_matches.is_empty():
        derniers_matches = dernier_match
    else:
        derniers_matches = pl.concat([derniers_matches, dernier_match], how="vertical")

In [9]:
derniers_matches.write_csv(output_path)